In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import re
import requests
from bs4 import BeautifulSoup

import time
from pathlib import Path

# GetDate
from datetime import datetime #,  date

In [2]:
#Reimportando arquivo - caso execucao tenha sido pausada
path_to_file = 'ListagemExport_teste00.csv'

listagem = pd.read_csv(path_to_file,
                       usecols= ['Projeto','Ano','Tipo','Ementa','Situacao','Autor','Codigo_Tipo'
                                 ,'Id_Projeto_Camara','Data_Projeto','Tramitacao_Encerrada','Aprovado'])

#Substitundo valores
listagem['Aprovado'] = listagem['Aprovado'].fillna('')
listagem['Tramitacao_Encerrada'] = listagem['Tramitacao_Encerrada'].fillna('')
listagem['Data_Projeto'] = listagem['Data_Projeto'].fillna('')
listagem['Codigo_Tipo'] = listagem['Codigo_Tipo'].fillna('')

listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].fillna(0)
listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].astype(int)

In [5]:
display(listagem)

,Projeto,Ano,Tipo,Ementa,Situacao,Autor,Codigo_Tipo,Id_Projeto_Camara,Data_Projeto,Tramitacao_Encerrada,Aprovado
0,11,2021,Requerimento,Requeiro a Excelentíssima Senhora Prefeita Mun...,Aprovado,Bejani Júnior,REQ,89111,,,
1,11,2021,Projeto de Resolução,Aprova as Contas da Prefeitura Municipal de Ju...,Transformado em Norma Jurídica,"André Luiz, Vagner de Oliveira, Nilton Militão",PRES,103039,,,


In [4]:
# ADICIONAR CONTROLE DE EXECUÇOES, FALHA E EXPORTAÇAO AUTOMATICA
# Preenchimento do campo Id_Projeto_Camara
count = 1
failed_req= 0
failed = False
id_projeto_camara = ''
max_failed_req = 0
notfound_count = 0
req_count = 0
max_req = 150
export = False

#https://www.camarajf.mg.gov.br/sal/listapropos.php?ano=2017&codtipo=PEDIF&numero=24

while count > 0:
    for i in listagem.index:
        if (listagem['Id_Projeto_Camara'][i] == '' or listagem['Id_Projeto_Camara'][i] == 0) and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'): 
            with requests.Session() as session:
                
                proposicao = False
                tipo = ''
                ementa = ''
                situacao = ''
                id_projeto_camara = 0
                failed = False

                # Variaveis de requisicao
                num_proj = listagem['Projeto'][i]
                ano_proj = listagem['Ano'][i]
                tipo_proj = listagem['Codigo_Tipo'][i]

                # Variaveis de resposta
                #status = ''
                
                try:
                    # Requisicao para o portal de propostas da Camara
                    query = {"numero": num_proj, "ano": ano_proj, "codtipo": tipo_proj}
                    resposta = session.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)
                    req_count+=1
                    
                    resp_format = BeautifulSoup(resposta.content, "html.parser")
                    fnd_proposicoes = resp_format.find('table', 
                                                        {'border':'0', 
                                                        'cellpadding':'0', 
                                                        'cellspacing':'0', 
                                                        'height':'100%', 
                                                        'width':'100%'}).table.tr.td.find_all('a', 
                                                                                                {'target':'blank'})

                    v_resultado = []
                    m_proposicoes = [[None]*6]*(int(len(fnd_proposicoes)/6))
                    
                    for f in fnd_proposicoes:
                        v_resultado.append(f)


                except:
                    #Assinala que houve falha
                    failed = True
                    time.sleep(180)
                    #print(resposta.status_code)
                    
                
                l, c, a = 0, 0, 0

                if v_resultado != []:
                    while a < len(v_resultado):
                        m_proposicoes[l][c] = v_resultado[a]
                        a+=1

                        # Testes - percorrendo os calores de linhas, colunas e contador
                        #print((a%6))
                        #print(f'a = {a}')
                        #print(f'l = {l}')
                        #print(f'c = {c}')

                        if ((a%6)) == 0:

                            tipo = str(re.search('>(.+?)<', (str(m_proposicoes[l][2]))).group(1))
                            ementa = str(re.search('block;">(.+?)<', (str(m_proposicoes[l][4]))).group(1))
                            situacao = str(re.search('>(.+?)<', (str(m_proposicoes[l][5]))).group(1))

                            if tipo == listagem['Tipo'][i] and ementa == listagem['Ementa'][i]:
                                id_projeto_camara = re.search('.PHP(.+?)TAR', str(m_proposicoes[0][0]).upper().replace(' ', '').replace('?NUM=','').replace('"','')).group(1)
                                situacao_proposicao = situacao
                                proposicao = True

                            l += 1
                            c = 0
                        else:
                            c += 1

                        if proposicao:
                            break
                
                else:
                    notfound_count += 1
                       
            session.close()
            time.sleep(0.5)
            listagem['Id_Projeto_Camara'][i] = id_projeto_camara 
            listagem['Situacao'][i] = situacao_proposicao
            
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve X falhas consecutivas ou houve mais de Y requisicoes consecutivas 
        if failed_req > max_failed_req or req_count > max_req:
            break

    if failed_req > max_failed_req or req_count > max_req:
        break  
  
        
    count = 0
    
    for i in listagem.index:
        if listagem['Id_Projeto_Camara'][i] == '' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
            count +=1


now = datetime.now()
dthr_finalizado = now.strftime("%d-%m-%Y_%H-%M-%S")

try:
#if True:
# Exportacao da listagem no estado atual para arquivo csv
    file = 'ListagemExport_checkpoint_' + dthr_finalizado
    #path_to_file = '../ListagemExp/'+file+'.csv'
    path_to_file = './'+file+'.csv'
    path = Path(path_to_file)

    exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Autor','Codigo_Tipo', 
                    'Id_Projeto_Camara', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

    if path.is_file():
        exp_header = False
    else:    
        exp_header = True

    listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

    export = True

#else:
except:
    export = False


if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados = {notfound_count}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')
elif req_count > max_req:
    print(f'Processo finalizado em {req_count} requisicoes. Nao encontrados = {notfound_count}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')
else:
    print(f'Fim do processo. Nao encontrados = {notfound_count}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')


#print(situacao)
#print(tipo) 
#print(ementa)
#print(id_projeto_camara)


Fim do processo. Nao encontrados = 0. Finalizado em: 03-12-2022_19-46-47. Status da exportacao: True.
